In [1]:
import pandas as pd

In [3]:
df2 = pd.read_excel('./data/STATION_COORDS_HACKATON.xlsx')
df2 = df2.interpolate()

df = pd.concat(pd.read_excel('./data/DISL_HACKATON.xlsx', sheet_name=None), ignore_index=True)

In [4]:
# sort by WAGNUM and OPERDATE
df = df.sort_values(by=['WAGNUM', 'OPERDATE'])

In [6]:
wagons = {}
trains = {}

# for i in range(50000):
for i in range(len(df)):
    try:
        wagon_id = int(df['WAGNUM'][i])
        start_station_id = int(df['ST_ID_DISL'][i])
        end_station_id = int(df['ST_ID_DEST'][i])
        train_idx = df['TRAIN_INDEX'][i]
    except:
        continue
    
    if not train_idx:
        continue

    try:
        train_start_station_id = int(train_idx.split('-')[0])
        train_id = int(train_idx.split('-')[1])
        train_end_station_id = int(train_idx.split('-')[2])
    except:
        # print(f'{train_idx}')
        continue

    ts = df['OPERDATE'][i]

    # check if station exists from pd2
    if start_station_id not in df2['ST_ID'].values:
        # print(f'{start_station_id}')
        continue

    if end_station_id not in df2['ST_ID'].values:
        # print(f'{end_station_id}')
        continue

    if wagon_id not in wagons:
        wagons[wagon_id] = {
            'train_id': train_id,
            'current_station_id': start_station_id,
            'target_station_id': end_station_id,
            'ts': ts
        }
    elif wagons[wagon_id]['ts'] < ts:
        if wagons[wagon_id]['current_station_id'] != start_station_id:
            wagons[wagon_id]['ts'] = ts

        wagons[wagon_id]['current_station_id'] = start_station_id
        wagons[wagon_id]['target_station_id'] = end_station_id

    if train_id not in trains:
        trains[train_id] = {
            'start_station_id': train_start_station_id,
            'current_station_id': start_station_id,
            'end_station_id': train_end_station_id,
            'ts': ts
        }
    elif trains[train_id]['ts'] < ts:
        trains[train_id]['start_station_id'] = train_start_station_id
        trains[train_id]['current_station_id'] = start_station_id
        trains[train_id]['end_station_id'] = train_end_station_id
        trains[train_id]['ts'] = ts

In [7]:
wagons[9902]

{'train_id': 829,
 'current_station_id': 11094,
 'target_station_id': 11137,
 'ts': Timestamp('2023-08-31 16:06:00')}

In [8]:
# find records where wagon id == 9902
df[df['WAGNUM'] == 9902]

,WAGNUM,OPERDATE,ST_ID_DISL,ST_ID_DEST,TRAIN_INDEX
11200075,9902,2023-06-08 23:03:00,1770,1770.0,2879-290-1770
11200000,9902,2023-06-08 23:03:01,1770,1770.0,2879-290-1770
11200139,9902,2023-06-08 23:04:00,1770,1770.0,2879-290-1770
11200084,9902,2023-06-08 23:05:00,1770,1770.0,2879-290-1770
3543917,9902,2023-06-09 03:42:00,1770,14766.0,2879-290-1770
...,...,...,...,...,...
3183827,9902,2023-08-31 11:35:00,11094,11137.0,10686-920-11094
3183826,9902,2023-08-31 12:47:00,11094,11137.0,10686-920-11094
3183825,9902,2023-08-31 13:34:00,11094,11137.0,10686-920-11094
3183823,9902,2023-08-31 15:42:00,11094,11137.0,10686-920-11094


In [9]:
trains[829]

{'start_station_id': 22690,
 'current_station_id': 2879,
 'end_station_id': 2879,
 'ts': Timestamp('2023-08-31 21:25:00')}

In [10]:
from models import Train

train_models = []

for id, train in trains.items():
    model = Train(
        id=id,
        **train
    )
    train_models.append(model)
    

In [11]:
from models import Wagon

wagon_models = []

for id, wagon in wagons.items():
    model = Wagon(
        id=id,
        **wagon
    )
    wagon_models.append(model)

In [12]:
from controller import Session, engine
from models import Base

Base.metadata.create_all(engine)

In [13]:
Session.add_all(train_models)
Session.add_all(wagon_models)

Session.commit()